In [1]:
import pymongo
import pandas as pd
from pandas import Series,DataFrame
import matplotlib as plt
import matplotlib.dates as mdate
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
#二行即可搞定画图中文乱码
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
pd.set_option('display.width', None)  # 设置字符显示宽度
pd.set_option('display.max_rows', None)  # 设置显示最大行
pd.set_option('display.max_columns', None)  # 设置显示最大行

#连接数据库
client = pymongo.MongoClient('localhost',27017)
futures = client.futures
market = futures.market
unit = futures.unit
position = futures.position

date='20181129'
#加载数据
market = DataFrame(list(market.find({'date': {'$gte':date}})))
unit = DataFrame(list(unit.find()))
position = DataFrame(list(position.find({'date': {'$gte': date}})))

# #列名重名
# position= position.rename(columns={'var':'variety'})

#删除/选取某行含有特殊数值的列

position=position.dropna()

# ##选择需要显示的字段
marketdata=market[['variety','date','change']]
position=position[['date','variety','symbol','long_openIntr','short_openIntr']]

#修改日期格式，非常方便
marketdata['date'] = pd.to_datetime(marketdata['date'], format='%Y/%m/%d')
position['date'] = pd.to_datetime(position['date'], format='%Y/%m/%d')
# 值替换replace()
# position=position.replace(['PTA'],'TA')
#汇总合约
netPosition=position.groupby(['date','variety'])[['long_openIntr','short_openIntr']].sum()

netPosition['净持仓']=netPosition.apply(lambda x:x['long_openIntr']-x['short_openIntr'],axis=1)
netPosition['上一日净持仓']=netPosition.groupby('variety')['净持仓'].shift(1)
netPosition['净持仓变化量']=netPosition.apply(lambda x: x['净持仓']-x['上一日净持仓'],axis=1)
netPosition=netPosition.dropna()

marketdata.set_index(['date','variety'])
#计算价格变化量

print(marketdata.head())
print(position.head())
# 并集
trade=pd.merge(netPosition,marketdata, on=['date','variety'],how='outer')
trade = trade.dropna(axis=0, how='any')
trade['交易信号'] = trade.apply(lambda x: 0 if x['净持仓变化量']*x['change']>=0  else 1 if x['净持仓变化量']>0 else -1,axis=1)
trade=trade.set_index('date')
trade = trade[trade['交易信号'] != 0]
del trade['上一日净持仓']
print(trade.head())


# trade.to_csv(r"c:\futures1.csv",mode='a',encoding='ANSI',header=True)

  variety       date  change
0       A 2018-11-29     4.0
1      AG 2018-11-29    37.0
2      AL 2018-11-29   -22.0
3      AP 2018-11-29  -358.0
4      AU 2018-11-29     2.0
        date variety symbol  long_openIntr  short_openIntr
0 2018-11-29      AP     AP         9115.0          9236.0
1 2018-11-29      AP     AP         7504.0          6772.0
2 2018-11-29      AP     AP         6457.0          5720.0
3 2018-11-29      AP     AP         5105.0          5698.0
4 2018-11-29      AP     AP         5081.0          5626.0
           variety  long_openIntr  short_openIntr      净持仓  净持仓变化量  change  \
date                                                                         
2018-11-30       A        66515.0         90136.0 -23621.0  -222.0    25.0   
2018-11-30       A        66515.0         90136.0 -23621.0  -222.0    25.0   
2018-11-30      AL       260597.0        258506.0   2091.0  8334.0   -76.0   
2018-11-30      AL       260597.0        258506.0   2091.0  8334.0   -76.0   
2018

e:\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
